# Importing necessary libraries

In [1]:
import joblib
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

In [4]:
import joblib

# Load the saved model
dt_model = joblib.load('best_decision_tree_model.pkl')

# Load feature encoders (dictionary of LabelEncoders)
label_encoders = joblib.load('feature_label_encoders.pkl')

# Load target encoder
target_encoder = joblib.load('incident_grade_encoder.pkl')


In [3]:
test_data=pd.read_csv("Microsoft_test_data.csv")
test_data.head()

,AlertTitle,Hour,Day,AccountName,Category,EntityType,CountryCode,DeviceName,City,State,IncidentGrade
0,563,22,4,863,LateralMovement,User,242,153085,10630,1445,BenignPositive
1,2,12,3,453297,CommandAndControl,Machine,242,2833,10630,1445,BenignPositive
2,10807,3,8,453297,LateralMovement,Process,242,153085,10630,1445,BenignPositive
3,0,12,12,453297,InitialAccess,CloudLogonSession,242,153085,10630,1445,FalsePositive
4,18,17,6,136104,Discovery,User,242,153085,10630,1445,BenignPositive


In [5]:
# Separate features and target
X_test = test_data.drop('IncidentGrade', axis=1)
y_test = test_data['IncidentGrade']

# Identify categorical columns (same as training)
cat_cols = X_test.select_dtypes(include=['object', 'category']).columns.tolist()

# Apply saved label encoders to each categorical column
for col in cat_cols:
    if col in label_encoders:
        le = label_encoders[col]
        X_test[col] = le.transform(X_test[col].astype(str))
    else:
        print(f"No encoder found for column: {col}")


In [6]:
# Encode target labels using saved encoder
y_test_encoded = target_encoder.transform(y_test)


In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Make predictions
y_pred_encoded = dt_model.predict(X_test)

# Decode predictions back to original labels
y_pred = target_encoder.inverse_transform(y_pred_encoded)

# Decode true labels for comparison (optional, for display/reporting)
y_test_original = y_test  # Already in original form

# Evaluate
print("Accuracy:", accuracy_score(y_test_encoded, y_pred_encoded))
print("\n Classification Report:\n", classification_report(y_test_original, y_pred))
print("\n Confusion Matrix:\n", confusion_matrix(y_test_original, y_pred))


Accuracy: 0.7753957419580161

 Classification Report:
                 precision    recall  f1-score   support

BenignPositive       0.76      0.84      0.80   1630942
 FalsePositive       0.70      0.63      0.66    868897
  TruePositive       0.83      0.79      0.81   1422856

      accuracy                           0.78   3922695
     macro avg       0.77      0.75      0.76   3922695
  weighted avg       0.78      0.78      0.77   3922695


 Confusion Matrix:
 [[1367789  140985  122168]
 [ 218300  547761  102836]
 [ 205146   91619 1126091]]
